### 导入数据

#### 得到CaseStudy需要的数据

In [9]:
import pandas as pd
Cdata = pd.read_csv("data/Drug_Protein_Num.csv",header=None)
Cdata.head()

,0,1
0,0,984
1,0,985
2,0,986
3,1,987
4,1,988


In [10]:
Pr_name = pd.read_csv('data/protein_name.csv')
Pr_name.head()

,Unnamed: 0,protein
0,984,9606.ensp00000364709
1,985,9606.ensp00000364731
2,986,9606.ensp00000233838
3,987,9606.ensp00000410294
4,988,9606.ensp00000393312


In [11]:
Pr_name = Pr_name.rename(columns={'Unnamed: 0': 'PrID'})
Pr_name.head()

,PrID,protein
0,984,9606.ensp00000364709
1,985,9606.ensp00000364731
2,986,9606.ensp00000233838
3,987,9606.ensp00000410294
4,988,9606.ensp00000393312


#### 输入所要研究的Drug编号

In [12]:
# 本次使用的Drug有：Risperidone db00734 275; Haloperidol db00502 182 
# Clozapine DB00363 120
# Pioglitazone DB01132 436
# Caffeine  DB00201 56
# drug_list = [275,182]
#drug_list = [56,120,436]
import numpy as np
drug_list = list(np.random.randint(0, 985, 985)) ## 选取全部进行训练
global Full_Case_data
Full_Case_data = pd.DataFrame()
for i in drug_list:
    Case_data= Cdata.loc[Cdata[0] == i]
    Case_data = Case_data.rename(columns={1: 'PrID'})
    dd2 = pd.merge(Pr_name,Case_data,how='left',on='PrID')
    dd2 = dd2[~dd2[0].isin([i])]
    dd2 = dd2.drop(['protein',0],axis=1)
    dd2['DrugID']=i
    dd2.insert(0, 'DrugID', dd2.pop('DrugID'))
    Full_Case_data = pd.concat([Full_Case_data, dd2], ignore_index=True)
Full_Case_data.head()

,DrugID,PrID
0,320,984
1,320,985
2,320,986
3,320,987
4,320,988


In [13]:
Full_Case_data.to_csv('data/Casestudy_Num_all.csv',header=0,index=0)

### 计算GCN Feaure

#### 导入特征

In [14]:
node_features = pd.read_csv('data/Drug_Protein_Attribute.csv',header = None) 
node_features = node_features.iloc[:,1:]
node_features.head() 

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
0,-0.007599,-0.007655,-0.003703,-0.007203,0.002032,0.006483,0.005779,-0.008866,-0.00496,0.005712,...,-0.005631,0.002488,-0.013618,-0.008738,0.007777,0.005095,0.00378,-0.015584,-0.005586,0.004156
1,-0.007599,-0.007655,-0.003703,-0.007203,0.002032,0.006483,0.005779,-0.008866,-0.00496,0.005712,...,-0.005631,0.002488,-0.013618,-0.008738,0.007777,0.005095,0.00378,-0.015584,-0.005586,0.004156
2,-0.007599,-0.007655,-0.003703,-0.007203,0.002032,0.006483,0.005779,-0.008866,-0.00496,0.005712,...,-0.005631,0.002488,-0.013618,-0.008738,0.007777,0.005095,0.00378,-0.015584,-0.005586,0.004156
3,-0.007599,-0.007655,-0.003703,-0.007203,0.002032,0.006483,0.005779,-0.008866,-0.00496,0.005712,...,-0.005631,0.002488,-0.013618,-0.008738,0.007777,0.005095,0.00378,-0.015584,-0.005586,0.004156
4,1.180584,0.526513,0.793145,-0.245426,-0.220634,0.559357,1.059898,0.148331,-0.98174,-0.789687,...,0.003815,-0.553314,0.003378,0.335591,0.143528,-1.134055,0.62961,0.330191,0.377519,-0.798777


In [15]:
#### 设置标签
import numpy as np
DTIs= pd.read_csv('data/Drug_Protein_Num.csv',header = None) 
labels = pd.DataFrame(np.random.rand(max(DTIs[1])+1,1))#pd.DataFrame(np.random.rand(max(NLMI_num),1))
labels[0:max(DTIs[0])+1]=0
labels[max(DTIs[0])+1:]=1
labels = labels[0] # 提取节点标签列
labels[:5]

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: 0, dtype: float64

In [16]:
# 构建初始的邻接矩阵 #('./data/DDI_train'+str(i)+'.csv')
def load_file_as_Adj_matrix(filename):
    import scipy.sparse as sp
    DTIs_train = pd.read_csv(filename,header=None)
    if max(DTIs_train[1]) != 1618:
        relation_matrix = np.zeros((1618+1,1618+1))
    else:
        relation_matrix = np.zeros((max(DTIs_train[1]+1),max(DTIs_train[1]+1)))
    for i, j in np.array(DTIs_train):
        lnc, mi = int(i), int(j)
        relation_matrix[lnc, mi] = 1
    Adj = sp.csr_matrix(relation_matrix, dtype=np.float32)
    return Adj

In [17]:
# 构建GCN训练需要的邻接方阵
import scipy.sparse as sp
import torch
def load_data(adj,node_features,node_labels):
  features = sp.csr_matrix(node_features, dtype=np.float32)  # 储存为csr型稀疏矩阵
  # build symmetric adjacency matrix   论文里A^=(D~)^0.5 A~ (D~)^0.5这个公式
  # adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
  # 对于无向图，邻接矩阵是对称的。上一步得到的adj是按有向图构建的，转换成无向图的邻接矩阵需要扩充成对称矩阵
  features = normalize(features)
  adj = normalize(adj + sp.eye(adj.shape[0]))   # eye创建单位矩阵，第一个参数为行数，第二个为列数
  # 对应公式A~=A+IN
  # 分别构建训练集、验证集、测试集，并创建特征矩阵、标签向量和邻接矩阵的tensor，用来做模型的输入
  idx_train = range(500)
  idx_val = range(500, 660)
  idx_test = range(660, int(adj.shape[0]))  
  features = torch.FloatTensor(np.array(features.todense()))  # tensor为pytorch常用的数据结构
  labels = torch.LongTensor(np.array(node_labels))
  adj = sparse_mx_to_torch_sparse_tensor(adj)   # 邻接矩阵转为tensor处理
  idx_train = torch.LongTensor(idx_train)
  idx_val = torch.LongTensor(idx_val)
  idx_test = torch.LongTensor(idx_test)
  return adj, features, labels, idx_train, idx_val, idx_test  
def normalize(mx):
  """Row-normalize sparse matrix"""
  rowsum = np.array(mx.sum(1))  # 对每一行求和
  r_inv = np.power(rowsum, -1).flatten()  # 求倒数
  r_inv[np.isinf(r_inv)] = 0.  # 如果某一行全为0，则r_inv算出来会等于无穷大，将这些行的r_inv置为0
  r_mat_inv = sp.diags(r_inv)  # 构建对角元素为r_inv的对角矩阵
  mx = r_mat_inv.dot(mx)
  # 用对角矩阵与原始矩阵的点积起到标准化的作用，原始矩阵中每一行元素都会与对应的r_inv相乘，最终相当于除以了sum
  return mx

def accuracy(output, labels):
  preds = output.max(1)[1].type_as(labels) # 使用type_as(tesnor)将张量转换为给定类型的张量。
  correct = preds.eq(labels).double()  # 记录等于preds的label eq:equal
  correct = correct.sum()
  return correct / len(labels)

def sparse_mx_to_torch_sparse_tensor(sparse_mx):    # 把一个sparse matrix转为torch稀疏张量
  """
  numpy中的ndarray转化成pytorch中的tensor : torch.from_numpy()
  pytorch中的tensor转化成numpy中的ndarray : numpy()
  """
  sparse_mx = sparse_mx.tocoo().astype(np.float32)
  indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
  # 不懂的可以去看看COO性稀疏矩阵的结构
  values = torch.from_numpy(sparse_mx.data)
  shape = torch.Size(sparse_mx.shape)
  return torch.sparse.FloatTensor(indices, values, shape)

In [18]:
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

class GraphConvolution(Module):

    # 初始化层：输入feature，输出feature，权重，偏移
    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))  # FloatTensor建立tensor
        # 常见用法self.v = torch.nn.Parameter(torch.FloatTensor(hidden_size))：
        # 首先可以把这个函数理解为类型转换函数，将一个不可训练的类型Tensor转换成可以训练的类型parameter并将这个parameter
        # 绑定到这个module里面，所以经过类型转换这个self.v变成了模型的一部分，成为了模型中根据训练可以改动的参数了。
        # 使用这个函数的目的也是想让某些变量在学习的过程中不断的修改其值以达到最优化。
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
            # Parameters与register_parameter都会向parameters写入参数，但是后者可以支持字符串命名
        self.reset_parameters()

    # 初始化权重
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        # size()函数主要是用来统计矩阵元素个数，或矩阵某一维上的元素个数的函数  size（1）为行
        self.weight.data.uniform_(-stdv, stdv)  # uniform() 方法将随机生成下一个实数，它在 [x, y] 范围内
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    '''
    前馈运算 即计算A~ X W(0)
    input X与权重W相乘，然后adj矩阵与他们的积稀疏乘
    直接输入与权重之间进行torch.mm操作，得到support，即XW
    support与adj进行torch.spmm操作，得到output，即AXW选择是否加bias
    '''
    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        # torch.mm(a, b)是矩阵a和b矩阵相乘，torch.mul(a, b)是矩阵a和b对应位相乘，a和b的维度必须相等
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias 
        else:
            return output
#通过设置断点，可以看出output的形式是0.01，0.01，0.01，0.01，0.01，#0.01，0.94]，里面的值代表该x对应标签不同的概率，故此值可转换为#[0,0,0,0,0,0,1]，对应我们之前把标签onthot后的第七种标签

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'


In [19]:
import torch.nn as nn
import torch.nn.functional as F
class GCN(nn.Module):
  def __init__(self, nfeat, nhid, nclass, dropout):
    # 底层节点的参数，feature的个数；隐层节点个数；最终的分类数
    super(GCN, self).__init__()  #  super()._init_()在利用父类里的对象构造函数
    self.gc1 = GraphConvolution(nfeat, nhid)   # gc1输入尺寸nfeat，输出尺寸nhid
    self.gc2 = GraphConvolution(nhid, nclass)  # gc2输入尺寸nhid，输出尺寸ncalss
    self.dropout = dropout
    self.weight = Parameter(torch.FloatTensor(nfeat, nhid))  # FloatTensor建立tensor
    # 输入分别是特征和邻接矩阵。最后输出为输出层做log_softmax变换的结果
  def forward(self, x, adj):
    x = F.relu(self.gc1(x, adj))   # adj即公式Z=softmax(A~Relu(A~XW(0))W(1))中的A~
    x2 = F.dropout(x, self.dropout, training = self.training)  # x要dropout
    x2 = self.gc2(x2, adj)
    return F.log_softmax(x2, dim = 1), x2   #, x  # 参数dim=1表示对每一行求softmax，那么每一行的值加起来都等于1。

In [21]:
# Training settings
learning_rate = 0.01
weight_decay = 5e-4
epoch_num = 200
dropout = 0.02
#in_size = node_features  #设置输入层的维数
hi_size = 64 # 16 #设置隐藏层的维数

Adj = load_file_as_Adj_matrix('data/Drug_Protein_Num.csv')
adj, train_features, trian_labels, idx_train, idx_val, idx_test = load_data(Adj,node_features,labels)
model = GCN(nfeat=train_features.shape[1],
        nhid=hi_size,
        nclass= 16,#labels.max().item() + 1,
        dropout=dropout)
model.train()
global Emdebding_train, output
output, Emdebding_train = model(train_features, adj)
Emdebding_GCN = pd.DataFrame(Emdebding_train.detach().numpy())
GCN_A = pd.concat([Emdebding_GCN, node_features], axis=1)
GCN_A.to_csv('casestudy/data/Emdebding_GCN.csv', header=None,index=False)
del Adj, adj, train_features, trian_labels, idx_train, idx_val, idx_test, output
GCN_A.head()

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,-0.011319,-0.304855,-0.022537,0.087194,0.109445,-0.083194,-0.065568,-0.269615,0.055453,-0.022204,...,-0.005631,0.002488,-0.013618,-0.008738,0.007777,0.005095,0.00378,-0.015584,-0.005586,0.004156
1,-0.014741,-0.296850,-0.023544,0.082032,0.102421,-0.084746,-0.063508,-0.270648,0.052009,-0.010894,...,-0.005631,0.002488,-0.013618,-0.008738,0.007777,0.005095,0.00378,-0.015584,-0.005586,0.004156
2,-0.021635,-0.311339,-0.025001,0.071055,0.116763,-0.082646,-0.068157,-0.274070,0.046882,-0.005360,...,-0.005631,0.002488,-0.013618,-0.008738,0.007777,0.005095,0.00378,-0.015584,-0.005586,0.004156
3,0.000605,-0.302556,-0.033224,0.092538,0.108618,-0.089233,-0.052742,-0.272499,0.050293,-0.032407,...,-0.005631,0.002488,-0.013618,-0.008738,0.007777,0.005095,0.00378,-0.015584,-0.005586,0.004156
4,0.003028,-0.302128,-0.027671,0.099886,0.118802,-0.080713,-0.055088,-0.278225,0.057023,-0.026456,...,0.003815,-0.553314,0.003378,0.335591,0.143528,-1.134055,0.62961,0.330191,0.377519,-0.798777


#### 导入节点的GCN_Embedding

In [22]:
GCN_data = pd.read_csv('casestudy/data/Emdebding_GCN.csv',header=None)
GCN_data.head()
len(GCN_data)

1619

In [23]:
# 导入节点的Embedding_DeepWalk
E_filename = 'casestudy/data/Embedding_grarep.txt'
Embedding_Node2vec = pd.read_csv(E_filename,sep=' ',header=None)
Embedding_Node2vec = Embedding_Node2vec.sort_values(0,ascending=True)# 建立序号排序
Embedding_Node2vec.reset_index(drop=True, inplace=True) 
Embedding_Node2vec = Embedding_Node2vec.iloc[:,1:]
Embedding_Node2vec[:10]
print(len(Embedding_Node2vec))

FileNotFoundError: [Errno 2] No such file or directory: 'casestudy/data/Embedding_grarep.txt'

### 训练模型

#### 生成负样本

In [ ]:
def NegativeGenerate(LncDisease, AllRNA, AllDisease, num):
    # 负样本为全部的disease-rna（328*881）中随机抽取，未在内LncDisease即为负样本
    import random
    NegativeSample = []
    counterN = 0
    while counterN < len(num):  # 随机选出一个疾病rna对
        counterR = random.randint(0, len(AllRNA) - 1)
        counterD = random.randint(0, len(AllDisease) - 1)
        DiseaseAndRnaPair = []
        DiseaseAndRnaPair.append(AllRNA[counterR])
        DiseaseAndRnaPair.append(AllDisease[counterD])
        flag1 = 0
        counter = 0
        while counter < len(num):
            if DiseaseAndRnaPair == LncDisease[counter]:
                flag1 = 1
                break
            counter = counter + 1
        if flag1 == 1:
            continue
        flag2 = 0
        counter1 = 0
        while counter1 < len(NegativeSample):  # 在已选的负样本中没有，防止重复
            if DiseaseAndRnaPair == NegativeSample[counter1]:
                flag2 = 1
                break
            counter1 = counter1 + 1
        if flag2 == 1:
            continue
        if (flag1 == 0 & flag2 == 0):
            NamePair = []  # 生成对
            NamePair.append(AllRNA[counterR])
            NamePair.append(AllDisease[counterD])
            NegativeSample.append(NamePair)
            counterN = counterN + 1
    return NegativeSample
Dr = pd.read_csv('data/drug_name.csv',header=0,names=['id','name'])
Pr = pd.read_csv('data/protein_name.csv',header=0,names=['id','name'])
alldata = DTIs.append(Case_data) # 添加了测试集中关系对，保证所生产的负样本不包含测试集
NegativeSample = NegativeGenerate(alldata.values.tolist(),Dr['id'].values.tolist(),Pr['id'].values.tolist(),DTIs)
NegativeSample = pd.DataFrame(NegativeSample)
NegativeSample.to_csv('casestudy/data/NegativeSample.csv', header=None,index=False)
NegativeSample.head()


In [ ]:
data_vec = []
Negative = pd.read_csv('casestudy/data/NegativeSample.csv',header=None)
Negative[2] = Negative.apply(lambda x: 0 if x[0] < 0 else 0, axis=1)
DTIs[2] = DTIs.apply(lambda x: 1 if x[0] < 0 else 1, axis=1)
result = DTIs.append(Negative)
labels = result[2]
train_data = result[[0,1]]
for i,j in np.array(train_data):
    data_vec.append(np.hstack([np.hstack([np.array(GCN_data.iloc[i]),np.array(Embedding_Node2vec.iloc[i])]),np.hstack([np.array(GCN_data.iloc[j]),np.array(Embedding_Node2vec.iloc[j])])]))
del Negative, train_data
np.array(data_vec).shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import joblib
case_RandomF = RandomForestClassifier(n_jobs=-1)# 选择最优的K=3传入模型
print('开始训练')#训练模型L
case_RandomF.fit(np.array(data_vec), np.array(labels))
# 保存模型
joblib.dump(case_RandomF,'casestudy/XGB.pkl')
print('保存完毕')

### 数据向量化

#### 导入test数据

In [ ]:
import pandas as pd
case_data = pd.read_csv('casestudy/data/Casestudy_Num_all.csv',header=None)
case_data.head()

In [ ]:
import numpy as np
Case_vectort = []
for i,j in np.array(case_data):#case_data
    Case_vectort.append(np.hstack([np.hstack([np.array(GCN_data.iloc[i]),np.array(Embedding_Node2vec.iloc[i])]),np.hstack([np.array(GCN_data.iloc[j]),np.array(Embedding_Node2vec.iloc[j])])]))
np.array(Case_vectort).shape

### 导入模型

In [ ]:
import joblib
model = joblib.load('casestudy/XGB.pkl')
y_score_GBDT = model.predict_proba(np.array(Case_vectort))
print('预测完毕！')

In [ ]:
y_score_GBDT

### 保存数据

In [ ]:
# ID 转 Name
import pandas as pd
drug_name = pd.read_csv('data/drug_name.csv')
drug_name = drug_name.rename(columns={'Unnamed: 0': 'DrugID'})
drug_name.head()

In [ ]:
protein_name = pd.read_csv('data/protein_name.csv')
protein_name = protein_name.rename(columns={'Unnamed: 0': 'PrID'})
protein_name.head()

In [ ]:
col_name = ['DrugID','PrID']
case_data.columns = col_name
case_data.head()

In [ ]:
dd = pd.merge(case_data,protein_name,how='left',on='PrID')
dd.head()

In [ ]:
dd2 = pd.merge(dd,drug_name,how='left',on='DrugID')
dd2.head()

In [ ]:
drug = pd.read_csv('casestudy/data/drug.csv')
drug.head()

In [ ]:
dd2['prob'] = y_score_GBDT[:,1]
dd2[['drug','protein','prob']].to_csv('casestudy/results/CaseStudy_Prob_all.csv', index = False)
print("保存完毕！")

#### 匹配蛋白质命名

In [ ]:
import pandas as pd
dd = pd.read_csv('casestudy/results/CaseStudy_Prob_all.csv')
dd.head()

In [ ]:
df = pd.read_excel('casestudy/data/uniprot-yourlist.xlsx')
df = df.rename(columns={'yourlist:M20210116A94466D2655679D1FD8953E075198DA82FDBA03': 'protein'})
df

In [ ]:
ddf = pd.merge(dd,df,how='left',on='protein')
ddf.head()

In [ ]:
ddf[['drug','protein','Entry name','Gene names','Protein names','prob']].to_csv('casestudy/results/NewCaseStudy_Prob_all.csv', index = False)